In [1]:
# Principal Component Regression (PCR)
#  Principal Component Regression (PCR) is a statistical technique for regression analysis that is used to reduce 
#     the dimensionality of a dataset by projecting it onto a lower-dimensional subspace. 

In [55]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np


data = pd.read_csv("C:/Users/Derya/Downloads/Team_Basketball.csv")
df = data.copy()

In [56]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
AtBat,322.0,380.928571,153.404981,16.0,255.25,379.5,512.00,687.0
Hits,322.0,101.024845,46.454741,1.0,64.00,96.0,137.00,238.0
HmRun,322.0,10.770186,8.709037,0.0,4.00,8.0,16.00,40.0
Runs,322.0,50.909938,26.024095,0.0,30.25,48.0,69.00,130.0
RBI,322.0,48.027950,26.166895,0.0,28.00,44.0,64.75,121.0
Walks,322.0,38.742236,21.639327,0.0,22.00,35.0,53.00,105.0
Years,322.0,7.444099,4.926087,1.0,4.00,6.0,11.00,24.0
CAtBat,322.0,2648.683230,2324.205870,19.0,816.75,1928.0,3924.25,14053.0
CHits,322.0,717.571429,654.472627,4.0,209.00,508.0,1059.25,4256.0
CHmRun,322.0,69.490683,86.266061,0.0,14.00,37.5,90.00,548.0


In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322 entries, 0 to 321
Data columns (total 20 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   AtBat      322 non-null    int64  
 1   Hits       322 non-null    int64  
 2   HmRun      322 non-null    int64  
 3   Runs       322 non-null    int64  
 4   RBI        322 non-null    int64  
 5   Walks      322 non-null    int64  
 6   Years      322 non-null    int64  
 7   CAtBat     322 non-null    int64  
 8   CHits      322 non-null    int64  
 9   CHmRun     322 non-null    int64  
 10  CRuns      322 non-null    int64  
 11  CRBI       322 non-null    int64  
 12  CWalks     322 non-null    int64  
 13  League     322 non-null    object 
 14  Division   322 non-null    object 
 15  PutOuts    322 non-null    int64  
 16  Assists    322 non-null    int64  
 17  Errors     322 non-null    int64  
 18  Salary     263 non-null    float64
 19  NewLeague  322 non-null    object 
dtypes: float64

In [58]:
dms = pd.get_dummies(df[['League','Division','NewLeague']])

In [59]:
dms

,League_A,League_N,Division_E,Division_W,NewLeague_A,NewLeague_N
0,1,0,1,0,1,0
1,0,1,0,1,0,1
2,1,0,0,1,1,0
3,0,1,1,0,0,1
4,0,1,1,0,0,1
...,...,...,...,...,...,...
317,0,1,1,0,0,1
318,1,0,1,0,1,0
319,1,0,0,1,1,0
320,1,0,1,0,1,0


In [60]:
y = df["Salary"]

In [61]:
X = df.drop(["Salary","League","Division","NewLeague"], axis=1).astype("float")

In [62]:
X.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,PutOuts,Assists,Errors
0,293.0,66.0,1.0,30.0,29.0,14.0,1.0,293.0,66.0,1.0,30.0,29.0,14.0,446.0,33.0,20.0
1,315.0,81.0,7.0,24.0,38.0,39.0,14.0,3449.0,835.0,69.0,321.0,414.0,375.0,632.0,43.0,10.0
2,479.0,130.0,18.0,66.0,72.0,76.0,3.0,1624.0,457.0,63.0,224.0,266.0,263.0,880.0,82.0,14.0
3,496.0,141.0,20.0,65.0,78.0,37.0,11.0,5628.0,1575.0,225.0,828.0,838.0,354.0,200.0,11.0,3.0
4,321.0,87.0,10.0,39.0,42.0,30.0,2.0,396.0,101.0,12.0,48.0,46.0,33.0,805.0,40.0,4.0


In [63]:
x_ = pd.concat([X, dms[["League_N","Division_W","NewLeague_N"]]], axis=1)
x_.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,PutOuts,Assists,Errors,League_N,Division_W,NewLeague_N
0,293.0,66.0,1.0,30.0,29.0,14.0,1.0,293.0,66.0,1.0,30.0,29.0,14.0,446.0,33.0,20.0,0,0,0
1,315.0,81.0,7.0,24.0,38.0,39.0,14.0,3449.0,835.0,69.0,321.0,414.0,375.0,632.0,43.0,10.0,1,1,1
2,479.0,130.0,18.0,66.0,72.0,76.0,3.0,1624.0,457.0,63.0,224.0,266.0,263.0,880.0,82.0,14.0,0,1,0
3,496.0,141.0,20.0,65.0,78.0,37.0,11.0,5628.0,1575.0,225.0,828.0,838.0,354.0,200.0,11.0,3.0,1,0,1
4,321.0,87.0,10.0,39.0,42.0,30.0,2.0,396.0,101.0,12.0,48.0,46.0,33.0,805.0,40.0,4.0,1,0,1


In [64]:
from sklearn.model_selection import train_test_split, cross_val_score,cross_val_predict
X_Train, X_Test,Y_Train,Y_Test = train_test_split(X,y, test_size=0.25, random_state=42)

In [65]:
print("X_Train:", X_Train.shape)
print("X_Test:", X_Test.shape)
print("Y_Train:", Y_Train.shape)
print("Y_Test:", Y_Test.shape)
training = df.copy()

X_Train: (241, 16)
X_Test: (81, 16)
Y_Train: (241,)
Y_Test: (81,)


In [66]:
print("training", training.shape)

training (322, 20)


In [67]:
from sklearn.decomposition import PCA              # Principal Component Analysis
from sklearn.preprocessing import scale

pca = PCA()

In [68]:
X_reduced_train = pca.fit_transform(scale(X_Train))

In [69]:
X_reduced_train[:3,1:4]

array([[ 0.42715432, -0.71035343, -0.035466  ],
       [-0.2681366 ,  1.91893847, -0.03148493],
       [-2.56889992, -0.74184886, -0.36720998]])

In [70]:
np.cumsum(np.round(pca.explained_variance_ratio_, decimals=4)*100)[:10]

array([45.93, 72.02, 81.98, 87.6 , 91.89, 94.83, 96.51, 97.67, 98.43,
       99.04])

In [72]:
lm = LinearRegression()

In [73]:
pcr_model = lm.fit(X_reduced_train,Y_Train)

ValueError: Input y contains NaN.